# Ping Tutorial
<i>Adapted for use with FABRIC from [WebServerExample](https://groups.geni.net/geni/wiki/WebServerExample)
    
In this tutorial you will learn:
* How to build a functional web server
* The basics of distributed programming
* Client/Server structures
* Issues in building high performance servers

    
<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-3 at https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/)
* You are comfortable with coding in C or C++
  

## 1. Design the Experiment
In this section, be careful to do the instructions listed with **"Do this"**, as well as running the code blocks.

### 1.1 Reserve three virtual machines at any aggregate


<b>Do this:</b>
    Set bastion_username to your username in the block below.


In [ ]:
import os
# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_FABRIC_USERNAME>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+<INSERT_YOUR_BASTION_KEY>

# Set the keypair FABRIC will install in your slice.
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

#### Import the Fabric API

In [ ]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

#### Create slice

In [ ]:
slice_name = 'MyWebServerSlice'
site = 'MAX'

server_name = 'server'
node_1_name = 'node1'
node_2_name = 'node2'

network_service_name1='bridge1'
network_service_name2='bridge2'

server_nic1_name = 'server-nic1'
server_nic2_name = 'server-nic2'
node1_nic_name = 'node1-nic1'
node2_nic_name = 'node2-nic1'

image = 'default_ubuntu_20'
image_type = 'qcow2'
cores = 2
ram = 8
disk = 10

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(slice_name)
    
    # Add node
    server = slice.add_node(name=server_name, site=site)
    server.set_capacities(cores=8, ram=16, disk=32)
    server.set_image(image)
    ifaces1 = server.add_component(model='NIC_Basic', name=server_nic1_name).get_interfaces()[0] 
    ifaces2 = server.add_component(model='NIC_Basic', name=server_nic2_name).get_interfaces()[0] 
    
    # Add node
    node1 = slice.add_node(name=node_1_name, site=site)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    ifacen1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0] 
    
    # Add node
    node2 = slice.add_node(name=node_2_name, site=site)
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    ifacen2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0] 
    
    # Network
    net1 = slice.add_l2network(name=network_service_name1, interfaces=[ifaces1, ifacen1])
    net2 = slice.add_l2network(name=network_service_name2, interfaces=[ifaces2, ifacen2])

    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

#### Get the slice

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print(f"Slice: {slice.get_name()}, {slice.get_state()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")

### 1.2 Configure the Nodes

#### Get the topology

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   Interfaces        : {node.get_interfaces()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print()                
except Exception as e:
    print(f"Fail: {e}")

#### Choose data interface IP addresses

In [ ]:
server_ip1 = '10.20.30.40'
server_ip2 = '10.20.30.41'
node1_ip = '10.20.30.42'
node2_ip = '10.20.30.43'

#### Configure server node

In [ ]:
try:
    server = slice.get_node(name=server_name)        
    ifaces1 = server.get_interface(network_name=network_service_name1)  
    ifaces1.set_ip(ip=server_ip1, cidr="24")
    
    ifaces2 = server.get_interface(network_name=network_service_name2)  
    ifaces2.set_ip(ip=server_ip2, cidr="24")
    stdout, stderr = server.execute(f'ip addr show {ifaces1.get_os_interface()}')
    print (stdout)
    stdout, stderr = server.execute(f'ip addr show {ifaces2.get_os_interface()}')
    print (stdout)
    
    server.execute("wget https://github.com/GENI-NSF/geni-tutorials/raw/master/WebServer/webexample.tar.gz")
    server.execute("tar xvfz webexample.tar.gz")
    server.execute("sudo ./webexample/websrv_init.sh")
    server.execute("sudo cp webexample/website/* /var/www/")
    
except Exception as e:
    print(f"Error: {e}")

#### Configure nodes

In [ ]:
try:
    node1 = slice.get_node(name=node_1_name)        
    ifacen1 = node1.get_interface(network_name=network_service_name1)  
    ifacen1.set_ip(ip=node1_ip, cidr="24")
    
    stdout, stderr = node1.execute(f'ip addr show {ifacen1.get_os_interface()}')
    print (stdout)
    
    node1.execute("wget https://github.com/GENI-NSF/geni-tutorials/raw/master/WebServer/webexample.tar.gz")
    node1.execute("tar xvfz webexample.tar.gz")
    node1.execute("sudo .webexample/webcl_init.sh")
except Exception as e:
    print(f"Error: {e}")
    
try:
    node2 = slice.get_node(name=node_2_name)        
    ifacen2 = node2.get_interface(network_name=network_service_name2)  
    ifacen2.set_ip(ip=node2_ip, cidr="24")
    
    stdout, stderr = node2.execute(f'ip addr show {ifacen2.get_os_interface()}')
    print (stdout)
    
    node2.execute("wget https://github.com/GENI-NSF/geni-tutorials/raw/master/WebServer/webexample.tar.gz")
    node2.execute("tar xvfz webexample.tar.gz")
    node2.execute("sudo .webexample/webcl_init.sh")
except Exception as e:
    print(f"Error: {e}")

### 1.3 Get the ssh command

Add bastion ssh configuration to .ssh/config if it's not already added.

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print()                
except Exception as e:
    print(f"Fail: {e}")

## 2. Execute the experiment

From here on you will be working directly from the terminal.

### 2.1 Open terminal windows
Open a terminal window by clicking the blue "+" to open the launcher, selecting "Terminal," and repeating.

### 2.2 Start and stop the web server
In the original setup of your sliver there a webserver already installed and running on the "Server" host. As you implement your own webserver you might need to stop or start the installed webserver.
* To Stop the webserver run:
```
    sudo service apache2 stop
```
* To Start the webserver run:
```
    sudo service apache2 start
```
### 2.3 Command Line Web Transfers
Instead of using a web browser, you can also use command line tools for web transfers. To do this, follow these steps:
* Log in to Client1
* You can download the web page using this command, in this example, the address for the server will be the IP address:
```
    wget 10.20.30.40
```
* The above command only downloads the index.html file from the webserver. As we are going to see later a web page may include other web pages or objects such as images, videos etc. In order to force wget to download all dependencies of a page use the following options:
```
    wget -m -p 10.20.30.40
```
* This will produce a '10.20.30.40', server, with the following data structure. Run:
```
    ls 10.20.30.40
```
### 2.4 Build your own Server
* At a high level, a web server listens for connections on a socket (bound to a specific port on a host machine). Clients connect to this socket and use a simple text-based protocol to retrieve files from the server. For example, you might try the following command on Client1:
```
    % telnet server 80 
    GET /index.html HTTP/1.0
```
(Type two carriage returns after the "GET" command). This will return to you (on the command line) the HTML representing the "front page" of the web server that is running on the Server host.)

In our setup we are using the [Apache web server](https://httpd.apache.org/). The default document root for Apache on a host running Ubuntu is under /var/www.
* Login to the Server host and run:
```
    ls /var/www/
```
This should give you a similar structure to the directory structure you got when you downloaded the whole site with wget on the previous steps.

## 3. Finish!
Delete your slice

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")